In [1]:
import os
import torch
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io
from PIL import Image

class ImageDataset(Dataset):
    """Project Image dataset"""

    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        #image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 2]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [ ]:
dataset = ImageDataset(
    csv_file='../src/data/raw_labes.csv', 
    root_dir='../src/data/raw', 
    transform=transforms.ToTensor()
    )

In [ ]:
dataset.transform

In [ ]:
train_csv_file='../dataset/set_dataset/train_labes.csv'
train_root_dir='../dataset/set_dataset/train/'

annotations = pd.read_csv(train_csv_file)

In [ ]:
annotations

In [ ]:
len(annotations)

In [ ]:
img_path = os.path.join(root_dir, annotations.iloc[0, 0])
img_path

In [ ]:
image = io.imread(img_path)
type(image)

In [ ]:
y_label = torch.tensor(int(annotations.iloc[0, 2]))
y_label

In [ ]:
transform_data = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize([224,224]),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
            ])

In [ ]:
image = transform_data(image)
image.shape

In [ ]:
image = Image.open(img_path)
image

In [ ]:
train_path = '../dataset/set_dataset/train'
train_labels = '../dataset/set_dataset/train_labes.csv'
val_path = '../dataset/set_dataset/val'
val_labels = '../dataset/set_dataset/val_labes.csv'
test_path = '../dataset/set_dataset/test'
test_labels = '../dataset/set_dataset/test_labes.csv'


# Hyperparameters
num_classes = 5
learning_rate = 1e-3
batch_size = 6
num_epochs = 10

train_transform = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.Resize([224,224]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ])

test_val_transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ])

train_dataset = ImageDataset(
    csv_file=train_labels, 
    root_dir=train_path, 
    transform=train_transform
    )
val_dataset = ImageDataset(
    csv_file=val_labels, 
    root_dir=val_path, 
    transform=test_val_transform
    ) 

test_dataset = ImageDataset(
    csv_file=test_labels, 
    root_dir=test_path, 
    transform=test_val_transform
    ) 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))


In [ ]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))
print(len(train_dataset) + len(val_dataset) + len(test_dataset))

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torchsummary import summary
import torch.nn.functional as F

In [ ]:
def calc_input_dims():
        batch_data = torch.zeros((1, 3, 224, 224))
        batch_data = conv1(batch_data)
        batch_data = pool1(batch_data)
        batch_data = conv2(batch_data)
        batch_data = conv3(batch_data)
        batch_data = pool2(batch_data)
        batch_data = conv4(batch_data)
        batch_data = pool3(batch_data)

        return int(np.prod(batch_data.size()))

In [ ]:
num_classes = 5

conv1 = nn.Conv2d(3, 64, 5)
pool1 = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(64, 128, 5)
conv3 = nn.Conv2d(128, 64, 5)
pool2 = nn.MaxPool2d(2, 2)
conv4 = nn.Conv2d(64, 64, 5)
pool3 = nn.MaxPool2d(2, 2)

input_dims = calc_input_dims()

fc1 = nn.Linear(input_dims, num_classes)


In [ ]:
input_dims

In [ ]:
fc1

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):

    def __init__(self, num_classes=6):
        super(Net, self).__init__()
        self.num_classes = num_classes
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.conv3 = nn.Conv2d(128, 64, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(64, 64, 5)
        self.pool3 = nn.MaxPool2d(2, 2)

        #input_dims = self.calc_input_dims()

        self.fc1 = nn.Linear(64*23*23, self.num_classes)

    # Function to calculate the input dimension to Linear layer
    # TODO: Implement calculation depending on the network structure
#     def calc_input_dims(self):
#         batch_data = torch.zeros((1, 3, 224, 224))
#         batch_data = self.conv1(batch_data)
#         batch_data = self.pool1(batch_data)
#         batch_data = self.conv2(batch_data)
#         batch_data = self.conv3(batch_data)
#         batch_data = self.pool2(batch_data)
#         batch_data = self.conv4(batch_data)
#         batch_data = self.pool3(batch_data)

#         return int(np.prod(batch_data.size()))

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool2(x)

        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool3(x)
        #print(x.shape)
        x = x.view(-1, 64 * 23 *23)
        x = self.fc1(x)

        return x

In [4]:
net = Net().to(device)
print(net)
summary(net, (3, 224, 224))

Net(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=33856, out_features=6, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 220, 220]           4,864
         MaxPool2d-2         [-1, 64, 110, 110]               0
            Conv2d-3        [-1, 128, 106, 106]         204,928
            Conv2d-4         [-1, 64, 102, 102]         204,864
         MaxPool2d-5           [-1, 64, 51,

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 16 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)
print(net)
summary(net, (3, 224, 224))

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 220, 220]             456
         MaxPool2d-2          [-1, 6, 110, 110]               0
            Conv2d-3         [-1, 16, 106, 106]           2,416
         MaxPool2d-4           [-1, 16, 53, 53]               0
            Linear-5                  [-1, 120]       5,393,400
            Linear-6                   [-1, 84]          10,164
            Linear-7                    [-1, 5]             425
Total params: 5,406,861
Trai

In [5]:
# Define Data paths
train_path = '../dataset/set_dataset/train'
train_labels = '../dataset/set_dataset/train_labes.csv'
val_path = '../dataset/set_dataset/val'
val_labels = '../dataset/set_dataset/val_labes.csv'
test_path = '../dataset/set_dataset/test'
test_labels = '../dataset/set_dataset/test_labes.csv'

In [6]:
mean_val = [0.5, 0.5, 0.5]
std_val = [0.5, 0.5, 0.5]

In [7]:
train_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean_val, std_val),
])

test_val_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean_val, std_val),
])

In [8]:
train_data = ImageDataset(train_labels, train_path, train_transforms)
val_data = ImageDataset(val_labels, val_path, test_val_transforms)
test_data = ImageDataset(test_labels, test_path, test_val_transforms)

In [9]:
train_loader = DataLoader(train_data, batch_size=6, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=6, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=6, shuffle=True, num_workers=4)


In [10]:
train_size = len(train_data)
val_size = len(val_data)
test_size = len(test_data)

In [11]:
print(f"Train data size: {train_size}\nValidation data size: {val_size}\n\
Test data size: {test_size}\nTotal data size: {train_size + val_size + test_size}")


Train data size: 1519
Validation data size: 474
Test data size: 379
Total data size: 2372


In [12]:
import torch.optim as optim

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


In [13]:
epochs = 2
total_step = len(train_loader)
loss_list = []
acc_list = []


for epoch in range(epochs):
    print(epoch)
    print('-'*10)
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        # Back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        
print(acc_list)
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         #print(loss)
#         loss.backward()
#         #print(loss)
#         optimizer.step()
#         running_loss += loss.item()
#         #print(running_loss/20)
#         if (i+1) % 20 == 0:    # print every 20 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 20))
#             running_loss = 0.0
        

0
----------


NameError: name 'num_epochs' is not defined

In [ ]:
# epochs = 2

# for epoch in range(epochs):
#     print(epoch)
#     print('-'*10)
#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         #print(loss)
#         loss.backward()
#         #print(loss)
#         optimizer.step()
#         running_loss += loss.item()
#         #print(running_loss/20)
#         if (i+1) % 20 == 0:    # print every 20 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 20))
#             running_loss = 0.0

In [22]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        #print(labels)
        outputs = net(images)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        #print(_)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(correct)

print(f'Accuracy of the network on the {test_size} test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 379 test images: 22 %


In [31]:
epochs = 2

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        print('inputs.size():', inputs.size())
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print('loss.item():', loss.item())
        print('running_loss:', running_loss)
        running_loss += loss.item() * inputs.size(0)
        print(running_loss)
        break
    print('train_loader.dataset: ', train_loader.dataset)
    running_loss /= len(train_loader.dataset)
    print('Runnuing Loss: ', running_loss)
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0
            
#print('Finish Training')

inputs.size(): torch.Size([6, 3, 224, 224])
loss.item(): 1.583852767944336
running_loss: 0.0
9.503116607666016
train_loader.dataset:  <__main__.ImageDataset object at 0x7fd9f0312a60>
Runnuing Loss:  0.0062561662986609715
inputs.size(): torch.Size([6, 3, 224, 224])
loss.item(): 1.7713686227798462
running_loss: 0.0
10.628211736679077
train_loader.dataset:  <__main__.ImageDataset object at 0x7fd9f0312a60>
Runnuing Loss:  0.006996847752915785


In [ ]:
epochs = 2

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        print(labels.shape)
        #print(inputs)
        break

In [ ]:
model_path = '../models/my_simple_net.pth'

model = torch.load(model_path)

In [ ]:
model

In [ ]:
net.state_dict()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [ ]:
classes = train_data.classes

In [ ]:
classes = ('bolt', 'flange', 'lead_block', 'nut', 'pipe')

dataiter = iter(train_loader)
images, labels = dataiter.next()

In [ ]:
imshow(torchvision.utils.make_grid(images))
#print(' '.join('%5s' % classes[labels[j]] for j in range(5)))

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()

In [ ]:
imshow(torchvision.utils.make_grid(images))
#print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(5)))


In [ ]:
net = Net()
net.load_state_dict(torch.load(model_path))

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
import numpy as np

A = np.array([[56.0, 00.0, 4.4, 68.8],
              [1.2, 104.0, 52.0, 8.0],
              [1.8, 135.0, 99.0, 0.9]])

print(A)

In [ ]:
cal = A.sum(axis=0)
print(cal)

In [ ]:
percentage = 100 * A/cal.reshape(1,4)
print(percentage)

In [ ]:
B = np.array([[56.0, 00.0, 4.4, 68.8],
              [1.2, 104.0, 52.0, 8.0],
              [1.8, 135.0, 99.0, 0.9]])

In [ ]:
C = np.array([56.0, 00.0, 4.4, 68.8, 58.2])

In [ ]:
d = B * C

In [14]:
def train(model, optimizer, loss_func, train_loader, val_loader, epochs=20, device='cpu'):
    for epoch in range(1,epochs + 1):
        train_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
    
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.data.item() * inputs.size(0) # TOCHECK (is the batch size and userd to get the loss of a
                                                            # batch when batch size is not a factor of train_size)
        train_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0.0
        num_examples = 0.0
        for batch in val_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(outputs, dim=1), dim=1)[1], labels)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, train_loss,
        valid_loss, num_correct / num_examples))

In [15]:
train(model, optimizer,torch.nn.CrossEntropyLoss(), train_loader,val_loader, epochs=5, device=device)


Epoch: 1, Training Loss: 1.79, Validation Loss: 1.79, accuracy = 0.21
Epoch: 2, Training Loss: 1.79, Validation Loss: 1.79, accuracy = 0.21
Epoch: 3, Training Loss: 1.79, Validation Loss: 1.79, accuracy = 0.21
Epoch: 4, Training Loss: 1.79, Validation Loss: 1.79, accuracy = 0.21
Epoch: 5, Training Loss: 1.79, Validation Loss: 1.79, accuracy = 0.21


In [37]:
from PIL import Image, ImageFile

classes = ('bolt', 'flange', 'lead_block', 'nut', 'pipe', 'pipe')

img = Image.open('../dataset/set_dataset/test/bolt_108.jpg')
img = test_val_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

model.eval()
prediction = F.softmax(model(img), dim=1)
prediction = prediction.argmax()
print(prediction)
print(classes[prediction])

tensor(5, device='cuda:0')
pipe


In [38]:
x = torch.tensor([[1, 2, 3, 4],
                [5, 6, 7, 8]])

In [28]:
x

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [35]:
torch.unsqueeze(x, dim=2)

tensor([[[1],
         [2],
         [3],
         [4]],

        [[5],
         [6],
         [7],
         [8]]])

In [10]:
import time
name = f"model_{int(time.time())}"
print(name)

model_1613745848


time.time

In [23]:
date = time.strftime("%d%m%Y-")

In [26]:
timestamp = f"model_{date}{int(time.time())}.pth"

In [27]:
timestamp

'model_19022021-1613746269.pth'

In [28]:
date = time.strftime("%d%m%Y-")
NAME = f"model_{date}{int(time.time())}.pth"
PATH = 'models/'

In [29]:
import os
full_path = os.path.join(PATH, NAME)

In [30]:
full_path

'models/model_19022021-1613746410.pth'